In [3]:
import os
import pandas as pd
import numpy as np
import torch.utils.data as data
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from utils.dataset_utils import concatenate_prices_returns, create_rolling_window_ts
from loss_functions.SharpeLoss import SharpeLoss
from models.DLPO import DLPO

In [6]:
# neural network hyperparameters
input_size = 4 * 2
output_size = 4
hidden_size = 64
num_layers = 1

# optimization hyperparameters
learning_rate = 1e-4

# training hyperparameters
device = torch.device('cpu')
epochs = 10
batch_size = 10
shuffle = True
drop_last = True
num_timesteps_in = 50
num_timesteps_out = 1
test_ratio = 0.2

# relevant paths
source_path = os.getcwd()
inputs_path = os.path.join(source_path, "data", "inputs")

# prepare dataset
prices = pd.read_excel(os.path.join(inputs_path, "etfs-zhang-zohren-roberts.xlsx"))
prices.set_index("date", inplace=True)
returns = np.log(prices).diff().dropna()
prices = prices.loc[returns.index]
features = concatenate_prices_returns(prices=prices, returns=returns)
idx = features.index
returns = returns.loc[idx].values.astype('float32')
prices = prices.loc[idx].values.astype('float32')
features = features.loc[idx].values.astype('float32')  

# define train and test datasets
X_train, X_test, prices_train, prices_test = train_test_split(features, prices, test_size=test_ratio, random_state=1)
X_train, X_val, prices_train, prices_val = train_test_split(X_train, prices_train, test_size=test_ratio, random_state=1) 

X_train, prices_train = create_rolling_window_ts(features=X_train, 
                                                 target=prices_train,
                                                 num_timesteps_in=num_timesteps_in,
                                                 num_timesteps_out=num_timesteps_out)
X_val, prices_val = create_rolling_window_ts(features=X_val, 
                                             target=prices_val,
                                             num_timesteps_in=num_timesteps_in,
                                             num_timesteps_out=num_timesteps_out)
X_test, prices_test = create_rolling_window_ts(features=X_test, 
                                               target=prices_test,
                                               num_timesteps_in=num_timesteps_in,
                                               num_timesteps_out=num_timesteps_out)

# define data loaders
train_loader = data.DataLoader(data.TensorDataset(X_train, prices_train), shuffle=False, batch_size=batch_size, drop_last=drop_last)
val_loader = data.DataLoader(data.TensorDataset(X_val, prices_val), shuffle=False, batch_size=batch_size, drop_last=drop_last)
test_loader = data.DataLoader(data.TensorDataset(X_test, prices_test), shuffle=False, batch_size=batch_size, drop_last=drop_last)

In [7]:
# (1) model
model = DLPO(input_size=input_size,
             output_size=output_size,
             hidden_size=hidden_size,
             num_layers=num_layers,
             batch_first=True).to(device)

# (2) loss fucntion
lossfn = SharpeLoss()

# (3) optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# (4) training procedure
training_loss_values = []
model.train()
for epoch in range(epochs + 1):

    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, (X_batch, prices_batch) in pbar:
                
        # compute forward propagation
        weights_pred = model.forward(X_batch)

        # compute loss
        loss = lossfn(prices_batch, weights_pred, ascent=True)

        # compute gradients and backpropagate
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        pbar.set_description("Epoch: %d, sharpe (loss): %1.5f" % (epoch, loss.item() * -1))
    
        # Iterate over all (x,y) pairs in validation dataloader
    model.eval()

    with torch.no_grad():
        for i,  (X_batch, prices_batch) in enumerate(val_loader):
            # compute forward propagation
            weights_pred = model.forward(X_batch)

            # compute loss
            loss = lossfn(prices_batch, weights_pred, ascent=True)
        
            training_loss_values.append(loss.item() * -1)

training_loss_df = pd.DataFrame(training_loss_values, columns=["sharpe_ratio"])

Epoch: 10, sharpe (loss): 0.18400: 100%|██████████| 279/279 [00:03<00:00, 82.34it/s]


In [10]:
model.eval()

# Store for analysis
weights = []
prices = []

pbar = tqdm(enumerate(test_loader), total=len(test_loader))
for i, (X_batch, prices_batch) in pbar:
    
    optimizer.zero_grad()
    
    # compute forward propagation
    weights_pred = model.forward(X_batch)

    # compute loss
    loss = lossfn(prices_batch, weights_pred, ascent=True)
    
    # compute gradients and backpropagate
    loss.backward()
    optimizer.step()
    pbar.set_description("Test sharpe (loss): %1.5f" % (loss.item() * -1))

    # store predictions and true values
    prices.append(prices_batch)
    weights.append(weights_pred)

Test sharpe (loss): 0.24907: 100%|██████████| 83/83 [00:01<00:00, 76.38it/s]
